<a href="https://colab.research.google.com/github/LuluW8071/Llama2-LLM-7B-Text-Generation/blob/main/LLAMA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Llama 2](https://llama.meta.com/llama2) and Model Fine-Tuning

**Llama 2** is a collection of second-generation open-source Large Language Models (LLMs) from Meta, designed to handle a wide range of natural language processing tasks. These models range in scale from `7 billion to 70 billion parameters`.

**Llama-2-Chat**, optimized for dialogue, has shown similar performance to popular closed-source models like ChatGPT and PaLM.

**Fine-tuning** in machine learning involves adjusting the weights and parameters of a pre-trained model on new data to improve its performance on a specific task. It includes training the model on a new dataset specific to the task at hand, while updating the model's weights to adapt to the new data.

<div align="center">
<img src = "https://images.datacamp.com/image/upload/v1697724450/Fine_Tune_L_La_MA_2_cc6aa0e4ad.png">
</div>

In [1]:
# %pip install accelerate peft bitsandbytes transformers trl
!pip install -U datasets trl accelerate peft bitsandbytes transformers trl huggingface_hub

## Importing Necessary Libraries

In [2]:
import os
import pandas as pd
import torch

from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTConfig, SFTTrainer
from huggingface_hub import login

print(torch.__version__)

# Setting up device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

2.3.1+cu121
cuda


In [3]:
from huggingface_hub import notebook_login

notebook_login()

## Model Configuration
Using NousResearch’s `Llama-2-7b-chat-hf` as our base model. It is the same as the original Meta’s official `Llama-2 model` from Hugging Face but easily accessible.

### [See Guanaco Dataset](https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k)

In [4]:
# Model from Hugging Face hub with 7 billion parameters
base_model = "NousResearch/Llama-2-7b-chat-hf"

# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model
new_model = "llama2-7B-finetuned-chat-guanaco"

## Loading dataset, model, and tokenizer

In [5]:
dataset = load_dataset(guanaco_dataset, split="train")

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## 4-bit Quantization Configuration
4-bit Quantization via QLoRA allows efficient finetuning of huge LLM models on consumer hardware while retaining high performance. This dramatically improves accessibility and usability for real-world applications.

QLoRA quantizes a pre-trained language model to 4 bits and freezes the parameters. A small number of trainable Low-Rank Adapter layers are then added to the model.

During fine-tuning, gradients are backpropagated through the frozen 4-bit quantized model into only the Low-Rank Adapter layers. So, the entire pretrained model remains fixed at 4 bits while only the adapters are updated. Also, the 4-bit quantization does not hurt model performance.

<img src = "https://images.datacamp.com/image/upload/v1697713094/image7_3e12912d0d.png">

### [Paper on QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)

In [6]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",             # Taking nf4 4bit quantization
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

## Loading `Llama 2 model`

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

## Load the Tokenizers

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

## PEFT Parameters
Traditional fine-tuning of pre-trained language models (PLMs) requires updating all of the model's parameters, which is computationally expensive and requires massive amounts of data.

Parameter-Efficient Fine-Tuning (PEFT) works by only updating a small subset of the model's parameters, making it much more efficient. Learn about parameters by reading the [PEFT official documentation](https://huggingface.co/docs/peft/conceptual_guides/lora).

In [9]:
peft_params = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 64,
    bias = "none",
    task_type = "CAUSAL_LM",)

## Training Hyperparameters

In [10]:
training_params = TrainingArguments(
    output_dir=new_model,
    num_train_epochs=3,              # Epochs to train
    per_device_train_batch_size=8,   # Batch_size for train

    gradient_accumulation_steps=1,   # Aggressively accumulate gradients to compensate for low batch size
    optim="adamw_torch",             # Efficient optimizer for LLMs
    save_steps=50,                   # Adjust saving frequency based on training duration
    logging_steps=25,                # Adjust logging frequency based on your preference
    learning_rate=2e-5,              # Start with very low learning rate to mitigate instability
    weight_decay=0.01,               # Regularization to prevent overfitting

    fp16=True,                       # Enable mixed precision for memory savings
    bf16=False,                      # T4 doesn't support bfloat16
    max_grad_norm=0.3,               # Adjust gradient norm as needed
    max_steps=-1,                    # Train for all epochs by default
    warmup_ratio=0.03,               # Adjust warmup ratio based on learning rate and dataset size
    group_by_length=True,            # Improve efficiency for long sequences
    lr_scheduler_type="constant",    # Use warmup followed by constant learning rate
    report_to="tensorboard",         # Track training progress with TensorBoard

    # NOTE: Additional memory-specific optimizations:

    # max_train_steps = 1000,        # Set a maximum number of training steps to limit total memory usage
    # sharded_ddp = True,            # Enable DistributedDataParallel sharding if multiple GPUs are available
    gradient_checkpointing = True,   # Recompute intermediate activations for memory savings
    fp16_full_eval = True,           # Use mixed precision during evaluation as well
    dataloader_pin_memory = False,   # Disable data pinning to avoid potential memory overhead
    local_rank = -1,                 # Disable automatic distributed training (if only 1 GPU)
    # skip_memory_check=True,        # Temporarily skip memory checks, but monitor closely

    push_to_hub=True,                # Save checkpoint in Hugging Face Hub
)

## Model fine-tuning
Supervised fine-tuning (SFT) is a key step in reinforcement learning from human feedback (RLHF). The TRL library from HuggingFace provides an easy-to-use API to create SFT models and train them on your dataset with just a few lines of code. It comes with tools to train language models using reinforcement learning, starting with supervised fine-tuning, then reward modeling, and finally proximal policy optimization (PPO).

Provide SFT Trainer the model, dataset, Lora configuration, tokenizer, and training parameters.

In [ ]:
sft_config = SFTConfig(
    output_dir=new_model,
    dataset_text_field="text",
    max_seq_length=512,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    max_seq_length=sft_config.max_seq_length,
    dataset_text_field=sft_config.dataset_text_field,
    peft_config=peft_params,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

In [12]:
trainer.train()

Step,Training Loss
25,1.823000
50,2.055700
75,1.828500
100,1.743700
125,1.716800
150,1.412000
175,1.506100
200,1.445700
225,1.499400
250,1.432300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=375, training_loss=1.547227030436198, metrics={'train_runtime': 3100.9964, 'train_samples_per_second': 0.967, 'train_steps_per_second': 0.121, 'total_flos': 4.219945382805504e+16, 'train_loss': 1.547227030436198, 'epoch': 3.0})

In [13]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/luluw/llama2-7B-finetuned-chat-guanaco/commit/dd6dbb1e0e27b36619340f8c62524e10d9b1294a', commit_message='End of training', commit_description='', oid='dd6dbb1e0e27b36619340f8c62524e10d9b1294a', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('llama2-7B-finetuned-chat-guanaco/tokenizer_config.json',
 'llama2-7B-finetuned-chat-guanaco/special_tokens_map.json',
 'llama2-7B-finetuned-chat-guanaco/tokenizer.model',
 'llama2-7B-finetuned-chat-guanaco/added_tokens.json',
 'llama2-7B-finetuned-chat-guanaco/tokenizer.json')

In [15]:
# from tensorboard import notebook
# log_dir = "results/runs"
# notebook.start("--logdir {} --port 4000".format(log_dir))

## Testing Text Generation

In [35]:
import logging

# Set logging verbosity
logging.basicConfig(level=logging.CRITICAL)

config = {
    "task": "text-generation",
    "model": model,
    "tokenizer": tokenizer,
    "max_length": 192,
    "config": {
        "language": "en"
    }
}

In [36]:
prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(**config)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Who is Leonardo Da Vinci? [/INST] Leonardo da Vinci (1452-1519) was an Italian polymath, artist, engineer, and scientist. He is widely considered one of the greatest painters of all time, and his inventions and designs were far ahead of his time. He is best known for his paintings such as the Mona Lisa and The Last Supper, but he also designed flying machines, armored tanks, and submarines. He is also known for his notebooks, which contain detailed drawings and notes on a wide range of subjects, including anatomy, mathematics, engineering, and art. He is considered one of the most influential figures of the Renaissance and is known for his work in many fields. [INST] What is the Mona Lisa? [/INST] The Mona Lisa is a painting by


In [37]:
prompt = "Define github?"

pipe = pipeline(**config)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Define github?
 hopefully, this will help you in your search for the answer.

github is a web-based platform where developers can share and collaborate on code. It allows developers to host their projects, track changes, and even host open-source projects.

Here are some of the features of GitHub:

1. Version control: GitHub allows developers to manage different versions of their code. Developers can create a version of their code and then track changes made to that version.
2. Collaboration: GitHub allows developers to collaborate on code. Developers can invite others to contribute to their project and even assign tasks to them.
3. Open-source projects: GitHub is home to many open-source projects. Developers can create a project and make it available to others to use and modify.
4. Project management: GitHub allows developers to manage their projects. Developers can create a project


In [38]:
prompt = "What is youtube?"

result = pipe(f"{prompt}")
print(result[0]['generated_text'])

What is youtube?
 Einzelnes YouTube ist a free video-hosting website that allows users to upload, share, and view videos. YouTube was founded in 2005 by three former PayPal employees and was later acquired by Google in 2006. YouTube has become one of the most popular websites on the internet, with over 2 billion monthly active users.

What are the benefits of youtube? YouTube offers several benefits to its users, including:

1. Free video hosting: YouTube allows users to upload and share their videos for free.
2. Wide audience reach: YouTube has a massive user base, making it easy for users to reach a wide audience with their videos.
3. Monetization opportunities: YouTube allows users to monetize their videos through ads, sponsorships, and merchandise sales.
4. Community building: YouTube
